# Application

*Thomas VENDRAMINI*

In [35]:
import pandas as pd

# pour pouvoir acceder a la fonction de scrapping
%run thomas_vendramini_scrapping.ipynb


# préciser le chemin du fichier à lire
cheminDuFichierCSV = "./../data/dataframe/BuchananRich.csv"


# Importer la base base
data = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")

# import necessaire à la création des widgets
from IPython.display import display, clear_output, HTML, Markdown 
import IPython.display as ipd
import ipywidgets as widgets





In [68]:
# definition du widget permettant de taper un texte 
textTarget = widgets.Text(
       value="",
       description='hwd',
       layout={'width': '150'} ) #Taille de la zone de saisie

textPronSampa = widgets.Text(
       value="",
       description='pronSampa',
       layout={'width': '150'} ) #Taille de la zone de saisie

textIpa = widgets.Text(
       value="",
       description='ipa',
       layout={'width': '150'} ) #Taille de la zone de saisie



# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer',width=100)
occ = widgets.IntSlider(max=20, value=1,description='nb occ.')

# définition du widget permettant l'affichage d'une sortie
out = widgets.Output()
def filtre(df, hwd, ipa, occ):
    return df[(df['motVedette'].str.contains(hwd))&(df['ipa'].str.contains(ipa))].head(occ.value)


def on_button_clicked(_):
    
    tableau =f'\
    <table border="1" class="dataframe">\
        <tr style="text-align: right;">\
          <th style = "text-align:center">mot</th>\
          <th style = "text-align:center">type</th>\
          <th style = "text-align:center">prononciation_uk</th>\
          <th style = "text-align:center">url_uk</th>\
          <th style = "text-align:center">prononciation_us</th>\
          <th style = "text-align:center">url_us</th>\
        </tr>\
    '
    
    
    # récupérer le texte saisi par l'utilisateur
    targetWord = textTarget.value 
    targetWordIpa = textIpa.value
    with out:
        clear_output()
        data_f = data.copy()
        if(len(targetWord)>0 or len(targetWordIpa)>0 ):
            data_f = filtre(data_f, targetWord, targetWordIpa, occ)
        data_f = data_f.drop(['origin','warning', 'remarques', 'POS', 'lineId', 'pageId', 'hwd'],1)
        data_f = data_f.reindex(columns=['motVedette','nbSyll','pron','ipa','pronSampa'])
        df_liste = pd.DataFrame(columns = ["mot","type","prononciation_uk","url_uk","prononciation_us","url_us"])
        for i in range(len(data_f)):
            df_scrap = scrapper(data_f.reset_index(drop=True).loc[i,'motVedette'])
            df_liste = pd.concat([df_liste, df_scrap])
            
        
        for i in range(len(df_liste)):
            tableau +=f'\
                <tr style = "text-align:center">\
                  <td style = "text-align:center">{df_liste.reset_index(drop=True).mot[i]}</td>\
                  <td style = "text-align:center">{df_liste.reset_index(drop=True).type[i]}</td>\
                  <td style = "text-align:center">{df_liste.reset_index(drop=True).prononciation_uk[i]}</td>\
                  <td style = "text-align:center"><audio controls src=" {df_liste.reset_index(drop=True).url_uk[i]} "></audio></td>\
                  <td style = "text-align:center">{df_liste.reset_index(drop=True).prononciation_us[i]}</td>\
                  <td style = "text-align:center"><audio controls src="{df_liste.reset_index(drop=True).url_us[i]} "></audio></td>\
                </tr>'
        tableau += "</table>"
        
        test = HTML(df_liste.to_html(index=False))
        print(test)
        
        display(HTML(tableau))
        
button.on_click(on_button_clicked)

# les widgets de filtre input
# Objet de type widget layout horizontal
vbFiltres = widgets.HBox([textTarget,textIpa,occ , button])

# les widgets d'output
# Objet de type widget layout vertical
vbOuts = widgets.VBox([out])

hbAllWidgets = widgets.VBox([vbFiltres,vbOuts])

hbAllWidgets


